# Código Python para buscar Comentarios Youtube e Gerar os arquivos .csv no formato normalizado e com label 

In [1]:
import os
import pandas as pd
import tkinter as tk
from tkinter import ttk
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

def search_youtube_comments(api_key, video_ids, max_results=100):
    youtube = build('youtube', 'v3', developerKey=api_key)
    all_comments = []

    for video_id in video_ids:
        print(f"Buscando comentários para o vídeo: {video_id}")
        comments = []
        next_page_token = None

        while True:
            try:
                response = youtube.commentThreads().list(
                    part='snippet,replies',
                    videoId=video_id,
                    maxResults=max_results,
                    pageToken=next_page_token
                ).execute()

                for item in response.get('items', []):
                    comment = item['snippet']['topLevelComment']['snippet']
                    comments.append({
                        'video_id': video_id,
                        'comment_id': item['id'],
                        'author': comment['authorDisplayName'],
                        'url': f"https://www.youtube.com/watch?v={video_id}&lc={item['id']}",
                        'date': comment['publishedAt'],
                        'likes': comment.get('likeCount', 0),
                        'shares': 0,  # No direct field for shares in YouTube API for comments
                        'comments': comment.get('totalReplyCount', 0),
                        'content': comment['textDisplay'],
                        'service': 'YouTube',
                        'sentiment': 'NEUTRAL',  # Placeholder sentiment, update based on your sentiment analysis
                        'engagement': 0,  # Placeholder engagement, update as needed
                        **comment  # Include all additional comment fields
                    })

                    # Process replies
                    for reply in item.get('replies', {}).get('comments', []):
                        reply_snippet = reply['snippet']
                        comments.append({
                            'video_id': video_id,
                            'comment_id': reply['id'],
                            'author': reply_snippet['authorDisplayName'],
                            'url': f"https://www.youtube.com/watch?v={video_id}&lc={reply['id']}",
                            'date': reply_snippet['publishedAt'],
                            'likes': reply_snippet.get('likeCount', 0),
                            'shares': 0,  # No direct field for shares in YouTube API for comments
                            'comments': 0,  # Replies to replies not handled in this example
                            'content': reply_snippet['textDisplay'],
                            'service': 'YouTube',
                            'sentiment': 'NEUTRAL',  # Placeholder sentiment, update based on your sentiment analysis
                            'engagement': 0,  # Placeholder engagement, update as needed
                            **reply_snippet  # Include all additional reply fields
                        })

                next_page_token = response.get('nextPageToken')
                if not next_page_token:
                    break

            except HttpError as e:
                print(f"Erro ao buscar comentários para o vídeo {video_id}: {e}")

        all_comments.extend(comments)

    return all_comments

def convert_sentiment(sentiment):
    sentiment_mapping = {
        'NEUTRAL': 0,
        'POSITIVE': 1,
        'NEGATIVE': 2
    }
    return sentiment_mapping.get(sentiment, 0)

def save_comments_to_csv(comments, output_directory):
    comments_directory = os.path.join(output_directory, 'comentarios')
    os.makedirs(comments_directory, exist_ok=True)

    for index, comment in enumerate(comments):
        video_id = comment['video_id']
        service = comment['service']
        date = pd.to_datetime(comment['date']).strftime('%Y%m%d%H%M')
        sentiment = convert_sentiment(comment['sentiment'])
        index_formatted = str(index).zfill(7)

        filename = f"{service}_{video_id}_{date}_{sentiment}_{index_formatted}.csv"
        filepath = os.path.join(comments_directory, filename)

        df_single_row = pd.DataFrame([comment])
        df_single_row.to_csv(filepath, index=False)

def get_video_ids(directory):
    video_ids = []
    for filename in os.listdir(directory):
        if filename.startswith("YouTube") and filename.endswith(".csv"):
            df = pd.read_csv(os.path.join(directory, filename))
            if 'ID' in df.columns:
                video_ids.extend(df['ID'].unique())
    return video_ids

def run_code(api_key):
    output_directory = r'C:\madonna\split'
    video_ids = get_video_ids(output_directory)

    print(f"Total de vídeos encontrados: {len(video_ids)}")
    print("Processando vídeos em lotes de 50...")
    
    # Processar IDs dos vídeos em lotes de 50
    for i in range(0, len(video_ids), 50):
        batch_video_ids = video_ids[i:i+50]
        print(f"Iniciando busca de comentários para o lote de vídeos {i+1}-{i+len(batch_video_ids)}")
        comments = search_youtube_comments(api_key, batch_video_ids)
        print(f"Comentários encontrados para o lote de vídeos {i+1}-{i+len(batch_video_ids)}: {len(comments)}")
        save_comments_to_csv(comments, output_directory)
        print(f"Comentários salvos para o lote de vídeos {i+1}-{i+len(batch_video_ids)}")

def on_submit():
    api_key = api_key_entry.get()
    run_code(api_key)

root = tk.Tk()
root.title("YouTube Comments Downloader")

frame = ttk.Frame(root, padding="10")
frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

api_key_label = ttk.Label(frame, text="API Key:")
api_key_label.grid(row=0, column=0, sticky=tk.W)
api_key_entry = ttk.Entry(frame, width=30)
api_key_entry.grid(row=0, column=1, sticky=(tk.W, tk.E))

submit_button = ttk.Button(frame, text="Submit", command=on_submit)
submit_button.grid(row=1, column=0, columnspan=2)

root.mainloop()

Total de vídeos encontrados: 510
Processando vídeos em lotes de 50...
Iniciando busca de comentários para o lote de vídeos 1-50
Buscando comentários para o vídeo: -9Ns1CUzFV0
Buscando comentários para o vídeo: -9PAoIrM_FU
Buscando comentários para o vídeo: -A5w07L20zI
Buscando comentários para o vídeo: -d126ZTmccg
Buscando comentários para o vídeo: -nezkIL1H8Y
Buscando comentários para o vídeo: -nnHsFJR3o0
Buscando comentários para o vídeo: -ptvNYrBdH0
Buscando comentários para o vídeo: -qYl-rmPn90
Buscando comentários para o vídeo: -uLFhoF0m9k
Buscando comentários para o vídeo: -uLFhoF0m9k
Buscando comentários para o vídeo: 00bWogGWG6Q
Buscando comentários para o vídeo: 04CU5I3UO9w
Buscando comentários para o vídeo: 07OUm83n9Qs
Buscando comentários para o vídeo: 0ALxCP6hEiM
Buscando comentários para o vídeo: 0APvnV3IF7A
Buscando comentários para o vídeo: 0D5djw7dW6g
Buscando comentários para o vídeo: 0D5djw7dW6g
Buscando comentários para o vídeo: 0MG-MRxV_ng
Buscando comentários para 

: 

# Update de codigo para ajustar entendimento da ultima reunião de 20240613. #